# Worst Case Analysis of Voltage Divider

Three resistors, $R_1$ (10k +/- 5%), $R_2$ (30k +/- 1%), and $R_3$ (10k) are in series from $V_{in}$ (5V +100mV / -50mV) to ground. Determine the worst-case variation of $V_{out}=V_{in}\cdot R_3/(R_1+R_2+R_3)$ by either Monte Carlo or Extreme Value analysis.

Which analysis method to use is generally dependent on two factors:
1. How many variables are there? Extreme Value analysis is performed by brute force; any more than ~10 parameters may take a very long time to compute.
2. Is it likely that the worst case occurs at the extremes of the inputs, or somewhere in the middle? Monte Carlo will be the better choice if maximum resonance is sought, for example.

Functions can of course be nested; a Monte Carlo analysis could be fed into by an Extreme Value analysis, or vice-versa.

In [1]:
%reload_ext lab_black

In [2]:
import numpy as np
import worstcase as wca

In [3]:
wca.config.n = 2000  # default: 5000
wca.config.sigfig = 4  # default: 3

In [4]:
# unit (default: unitless) and tag (default: "") are optional
spd_initial = wca.param.bytol(nom=2, tol=0.1, rel=True, unit=wca.unit("m/s"), tag="v0")
accel = wca.param.byrange(nom=0.2, lb=0.1, ub=0.5, unit=wca.unit("m/s**2"), tag="a")
distance = wca.param.byrange(nom=1, lb=0.8, ub=1.1, unit=wca.unit.km, tag="dx")

In [5]:
print(spd_initial)
print(accel)
print(distance)

v0: 2 m/s (nom), 1.8 m/s (min), 2.2 m/s (max)
a: 200 mm/s² (nom), 100 mm/s² (min), 500 mm/s² (max)
dx: 1 km (nom), 800 m (min), 1.1 km (max)


In [6]:
# tag is take to be function.__name__ as the default
@wca.param.ev(spd_initial, accel.nom, distance)
def spd_final(v, a, x):
    return np.sqrt(v ** 2 + 2 * a * x)

In [7]:
spd_final

spd_final (ev)
├── dx
└── v0

In [8]:
spd_final()

spd_final: 20.1 m/s (nom), 17.98 m/s (min), 21.09 m/s (max)

In [9]:
spd_final_noaccel = spd_final(a=0 * wca.unit("m/s**2"), tag="spd_noaccel")

In [10]:
spd_final_noaccel

spd_noaccel (ev)
├── dx
└── v0

In [11]:
spd_final_noaccel()

spd_noaccel: 2 m/s (nom), 1.8 m/s (min), 2.2 m/s (max)

In [12]:
spd_relative = wca.param.bytol(
    nom=20, tol=1, rel=False, unit=wca.unit("mi/hr"), tag="vrel"
)

In [13]:
print(spd_relative)

vrel: 20 mi/hr (nom), 19 mi/hr (min), 21 mi/hr (max)


In [14]:
@wca.param.mc(spd_final, spd_relative)
def spd_total(vf, vr):
    return vf + vr

In [15]:
spd_total

spd_total (mc)
├── spd_final (ev)
│   ├── dx
│   └── v0
└── vrel

In [16]:
spd_total()

spd_total: 29.04 m/s (nom), 26.5 m/s (min), 30.42 m/s (max)

In [17]:
print(spd_total.ss(accel, tag="accel-sens")())
print(spd_total.ss(distance, tag="distance-sens")())
print(spd_total.ss([accel, distance], tag="accel/distance-sens")())

print(spd_total.ss(spd_final, tag="final-speed-sens")())
print(spd_total.ss(spd_relative, tag="rel-speed-sens")())

29.040551242241783 meter / second
distance-sens: 29.04 m/s (nom), 26.94 m/s (min), 30.01 m/s (max)
accel/distance-sens: 29.04 m/s (nom), 26.94 m/s (min), 30.01 m/s (max)
final-speed-sens: 29.04 m/s (nom), 26.92 m/s (min), 30.03 m/s (max)
rel-speed-sens: 29.04 m/s (nom), 28.59 m/s (min), 29.49 m/s (max)


In [18]:
spd_total.ss([accel, distance], tag="accel/distance-sens")

accel/distance-sens (mc)
└── spd_final (ev)
    └── dx

In [19]:
spd_total.ss(spd_final, tag="final-speed-sens")

final-speed-sens (mc)
└── spd_final (ev)
    ├── dx
    └── v0

In [20]:
spd_total.ss([spd_relative, spd_initial], tag="rel-speed-sens")

rel-speed-sens (mc)
├── spd_final (ev)
│   └── v0
└── vrel